In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime 
import duckdb 

In [30]:
## https://www.kaggle.com/datasets/irkaal/english-premier-league-results?resource=download
PREMIER_PATH = "data/results_premier/results.csv"
ML_PATH = "data/ml"

df = pd.read_csv(PREMIER_PATH, encoding = "ISO-8859-1")
con = duckdb.connect()


In [3]:
def add_records(row):
    home = {
        'season': row.Season,
        'team': row.HomeTeam,
        'opponent': row.AwayTeam,
        'datetime': get_datetime(row.DateTime),
        'veneau': 'home',
        'GF': row.FTHG,
        'GA': row.FTAG,
        'result': get_match_result(row.FTHG, row.FTAG),

    }

    away  = {
        'season': row.Season,
        'team': row.AwayTeam,
        'opponent': row.HomeTeam,
        'datetime': get_datetime(row.DateTime),
        'veneau': 'away',
        'GF': row.FTAG,
        'GA': row.FTHG,
        'result': get_match_result(row.FTAG, row.FTHG),

    }
    all_records.extend([home,away])

def get_datetime(str):
    date_str = str.strip('Z').replace('T',' ')
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

def get_match_result(GF, GA):
    if GF == GA:
        return 'D'
    if GA > GF:
        return 'L'
    else:
        return 'W'
        



In [27]:
all_records = list()
df.apply(lambda row: add_records(row), axis=1)
df_matches = pd.DataFrame(all_records)[["season","team","opponent","datetime","veneau","GF","GA","result"]]

In [28]:
sql = """
with matches as (
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY season, team ORDER BY datetime) AS match_number
FROM df_matches
ORDER BY season, datetime)
SELECT
    m.*,
    (SELECT COUNT(*) FROM matches WHERE season = m.season AND team = m.team AND result = 'W' AND match_number > m.match_number - 6 AND match_number < m.match_number) AS last_wins,
    (SELECT COUNT(*) FROM matches WHERE season = m.season AND team = m.team AND result = 'L' AND match_number > m.match_number - 6 AND match_number < m.match_number) AS last_loses,
    (SELECT COUNT(*) FROM matches WHERE season = m.season AND team = m.team AND result = 'D' AND match_number > m.match_number - 6 AND match_number < m.match_number) AS last_draws
FROM matches m
ORDER BY season, datetime;


"""
results = con.execute(sql).df()
results["home_code"] = results.team.astype("category").cat.codes
results["away_code"] = results.opponent.astype("category").cat.codes
results


,season,team,opponent,datetime,veneau,GF,GA,result,match_number,last_wins,last_loses,last_draws,home_code,away_code
0,1993-94,Arsenal,Coventry,1993-08-14 00:00:00,home,0,3,L,1,0,0,0,0,15
1,1993-94,Aston Villa,QPR,1993-08-14 00:00:00,home,4,1,W,1,0,0,0,1,34
2,1993-94,Blackburn,Chelsea,1993-08-14 00:00:00,away,2,1,W,1,0,0,0,4,14
3,1993-94,Chelsea,Blackburn,1993-08-14 00:00:00,home,1,2,L,1,0,0,0,14,4
4,1993-94,Coventry,Arsenal,1993-08-14 00:00:00,away,3,0,W,1,0,0,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221,2021-22,Crystal Palace,Leicester,2022-04-10 14:00:00,away,1,2,L,31,3,0,2,16,24
22222,2021-22,Leicester,Crystal Palace,2022-04-10 14:00:00,home,2,1,W,29,3,1,1,24,16
22223,2021-22,Norwich,Burnley,2022-04-10 14:00:00,home,2,0,W,31,0,4,1,30,11
22224,2021-22,Liverpool,Man City,2022-04-10 16:30:00,away,2,2,D,31,5,0,0,25,26


In [36]:
sql = """
select 
team as home,
opponent as away,
    home_code,
    away_code,
    last_wins,
    last_draws,
    last_loses,
    GF,
    GA
from results
where 
    GF < 10 and 
    GA < 10 


"""
df_history = con.execute(sql).df()


In [37]:
df_history.to_csv(f'{ML_PATH}/df_history.csv', index=False)
